<a href="https://colab.research.google.com/github/jaketerrito/speedchallenge/blob/master/Feedforward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/jaketerrito/speedchallenge/raw/master/data/train.mp4
!wget https://github.com/jaketerrito/speedchallenge/raw/master/data/train.txt

--2019-03-01 23:46:28--  https://github.com/jaketerrito/speedchallenge/raw/master/data/train.mp4
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/jaketerrito/speedchallenge/master/data/train.mp4 [following]
--2019-03-01 23:46:28--  https://media.githubusercontent.com/media/jaketerrito/speedchallenge/master/data/train.mp4
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130034541 (124M) [application/octet-stream]
Saving to: ‘train.mp4’

train.mp4           100%[===================>] 124.01M   156MB/s    in 0.8s    

2019-03-01 23:46:30 (156 MB/s) - ‘train.mp4’ saved [

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2 as cv

In [0]:
import numpy as np
import time
from os.path import splitext

def denseflow(fname, resize_factor):
    cap = cv.VideoCapture(fname)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
        exit()

    # Get the video dimensions, 3 is the ordinal value of CV_CAP_PROP_FRAME_WIDTH, 4 is CV_CAP_PROP_FRAME_HEIGHT
    # Also resize them because these images are too big
    width = int(cap.get(3) / resize_factor)
    height = int(cap.get(4) / resize_factor)
    shape = (width,height)
    size = int(cap.get(7))
    ret, frame1 = cap.read()
    frame1 = cv.resize(frame1,shape)
    prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255

    frames = np.zeros((size,hsv.shape[0],hsv.shape[1],3))
    frames[0] = hsv

    fcount = 1
    t = time.time()

    while(1):
        ret, frame2 = cap.read()
        if not ret:
            break
        frame2 = cv.resize(frame2,shape)
        next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
        flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        frames[fcount] = hsv
        prvs = next
        fcount += 1
    cap.release()

    f, ext = splitext(fname)
    #np.savez(f+'_op', frames)
    return frames

In [13]:
video = denseflow('train.mp4', 4)

KeyboardInterrupt: ignored

In [0]:
def data_generator(video, speeds, batch_size, sequence_length):
	while True:
		x = []
		y = []
		while len(x) < batch_size:
			frame_num = random.randrange(sequence_length,len(video))

			sequence = video[frame_num-sequence_length:frame_num]
			'''
			flip = random.choice([True,False])
			angle = random.uniform(-20,20)
			scale = random.uniform(.8,1.2)
			
			for i, image in enumerate(sequence):
				# Augmentation
				image = skimage.transform.rescale(image, scale=scale)
				image = skimage.transform.resize(image, output_shape=sequence[i].shape, mode='constant')
				image = skimage.transform.rotate(image, angle=angle)

				# Really need to see the types of values before we add this noise
				image = image + np.random.normal(scale=.5,size=image.shape)

				#normalize input!
				image = image / 255
				sequence[i] = image
			'''
			x.append(sequence)
			y.append(speeds[frame_num])
		yield np.array(x), np.array(y)

def prediction_generator(video, sequence_length):
	i = 0
	while True:

		yield np.array([video[i:i+sequence_length]])
		i += 1

In [10]:
print("Processing speeds.")
with open('train.txt') as f:
	speeds = f.readlines()
	speeds = np.array([float(x.strip()) for x in speeds])
	speeds = speeds

Processing speeds.


In [0]:
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv3D, MaxPooling3D, MaxPooling2D, BatchNormalization, Dropout, LeakyReLU, PReLU
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD, RMSprop

In [12]:
batch_size = 16
sequence_length = 8
epochs = 25
split = .9

width = video.shape[2]
height = video.shape[1]
video_size = len(video)

train_gen = data_generator(video[:int(video_size*split)], speeds[:int(video_size*split)], batch_size, sequence_length)
val_gen = data_generator(video[int(video_size*split):], speeds[int(video_size*split):], batch_size, sequence_length)
pred_gen = prediction_generator(video, sequence_length)

# Will return a feature and label set.	
# Features are a list of image sequences in the form: (sequence_length, img_height, img_width, dimensions)
inputs = Input((sequence_length,height,width,3))

# A convolution being applied to each image seperately
x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(inputs)
x = LeakyReLU(alpha=0.1)(x)
x = BatchNormalization()(x)
x = Conv3D(32,(3,3,3),strides=(2,2,2),activation=None)(x)
x = LeakyReLU(alpha=0.1)(x)
x = BatchNormalization()(x)
x = Conv3D(32,(3,3,3),strides=(2,2,2),activation=None)(x)
x = LeakyReLU(alpha=0.1)(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dropout(.5)(x)

x = Dense(32,activation=None)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(16,activation=None)(x)
x = LeakyReLU(alpha=0.1)(x)
outputs = Dense(1,activation=None)(x)
model = Model(inputs=inputs,outputs=outputs) 
model.compile(RMSprop(),loss='mean_squared_error')

model.summary()

NameError: ignored

In [0]:
history = model.fit_generator(
	train_gen, 
	steps_per_epoch=int(video_size*split/batch_size), 
	validation_data=val_gen, 
	validation_steps=int(video_size*(1-split)/batch_size),
	epochs=epochs,
	verbose=True,
	callbacks=[ModelCheckpoint('./data/weights.hdf5',save_best_only=True)])

In [0]:
model2 = load_model(filepath='./data/weights.hdf5')
model2.compile(RMSprop(),loss='mean_squared_error')

# Plot the training loss against the validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Loss', 'Validation Loss'])
plt.savefig(fname='./data/lossplot')
plt.show()

plt.clf()

# Plotting predicted speeds against real speeds
plt.plot(model2.predict_generator(pred_gen, steps=video_size-sequence_length))
plt.plot(speeds)
plt.xlabel('Frame')
plt.ylabel('Speed in mph')
plt.legend(['Predicted', 'Real'])
plt.savefig(fname='./data/speedplot')
plt.show()